In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from PIL import ImageGrab
import requests
import os
import subprocess
import time

In [37]:

class crawling:
    
    def __init__(self):
        #self.html = ""
        #self.soup = ""
        # 탐자를 우회하기 위함.(http1.1 통신 기준으로 http header에 user-Agent를 속인다.)
        self.header ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)\AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
                        "Accept":"text/html,application/xhtml+xml,application/xml;\
                        q=0.9,imgwebp,*/*;q=0.8"}

        
    def parsing(self,url):
        html = requests.get(url,headers=self.header)
        #파싱
        soup = BeautifulSoup(html.content,"html.parser")
        return soup 
    
        
    def Naver(self,start_date,end_date,keyword,sort):
        #해당 사이트 주소 리스트
        self.href = []
        self.company_name =[]
        self.select_user_company = []
        self.start_date = start_date
        self.end_date = end_date
        self.keyword = keyword
        
        #접근 오류 해결하기 위함.
        requests.packages.urllib3.disable_warnings()
        page = 1
        s_from = self.start_date.replace(".","")
        e_to = self.end_date.replace(".","")
        #sort 0 : 관련도,  1 : 최신순 , 2 : 오래된 
      
        max_page = (10-1)*10+1
        
        while page <=max_page:
            
            url =  "https://search.naver.com/search.naver?where=news&query="+keyword+ "&sort="+str(sort)+"&ds=" + self.start_date + "&de=" + self.end_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
            
            soup = self.parsing(url)
            
            new_title = soup.select("._sp_each_title")
            #뉴스 기사 주소 수집
            for urls in new_title:
                self.href.append(urls['href'])
           
            # 뉴스기사 회사 이름    
            company_names = soup.select("._sp_each_source ")
            for urls in company_names:
                #회사이름 뒤에 언론사 선정 붙음.
                self.company_name.append(urls.text.replace("언론사 선정",""))
             
            
            #네이버 페이지 전환 규칙
            page +=10
        
        for href,company in zip(self.href,self.company_name):
            #print(href,company)
            if company == "아시아경제":
                self.select_user_company.append(href)

       
 
        
        self.select_company()           

    
    def select_company(self):
        self.__options = webdriver.ChromeOptions()
        self.__options.add_argument('headless')
        self.__options.add_argument('window-size=1920x1080')
        self.__options.add_argument('--disable-gpu')
        self.__options.add_argument("lang=ko_KR") # 한국어!
        self.__options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
                
        #,options= self.__options        
        self.__driver = webdriver.Chrome('C:/Users/user/Downloads/chromedriver_win32/chromedriver',options= self.__options) 
        
        print(len(self.select_user_company))
        print(self.select_user_company)
        number =1
        
        for news in self.select_user_company:
            #self.__driver.implicitly_wait(3)
            self.__driver.get(news)
            
            #스크롤 down
            self.__driver.execute_script("window.scrollTo(0,2500);")
            iframes = self.__driver.find_elements_by_css_selector('iframe')
            
            
            #time.sleep(3)
                
            
            screenshot_name= "댓글"
            
            try:
                for iframe in iframes:
                        #print(iframe.get_attribute('title'))
                        if iframe.get_attribute('title') == 'livere':
                            
                            self.__driver.get(iframe.get_attribute("src"))
                            tt = self.__driver.find_elements_by_css_selector('div')
                            #ss = self.__driver.find_elements_by_css_selector('#wrapper > div.reply-count')
                            #print(ss.get_attribute('reply-count'))
                            ##wrapper > div.reply-count
                          
                            for i in tt:
                                if i.get_attribute('class') == 'reply-content-wrapper':
                                    lans = i.get_attribute('data-content')
                                    #ss = self.__driver.find_elements_by_css_selector('#wrapper > div.reply-count')
                                    #print(ss.get_attribute('reply-count'))
                                    #i.get_attribute('')
                                    self.__driver.implicitly_wait(1)
                                    #time.sleep(5)
                                    self.__driver.save_screenshot(screenshot_name+str(number)+'.png')
                                    number+=1
                                    print(i.get_attribute('data-content'))
            except:
                    continue
        
      
                    #break
                    
        #self.__driver.quit()
        
        
    @property   
    def get_select_user_company(self):
        
        return self.select_user_company
                                

            
            
        
        




In [36]:
naver =crawling()
#self,start_date,end_start,keyword,sort
naver.Naver('2019.7.00','2019.9.01','심석희','0')




4
['https://view.asiae.co.kr/article/2019083017304084385', 'https://view.asiae.co.kr/article/2019082310060528531', 'https://view.asiae.co.kr/news/view.htm?idxno=2019060321045386858', 'http://view.asiae.co.kr/news/view.htm?idxno=2019040810272026701']


In [ ]:
class seleniums:
    
    def __init__(self):
        #self.__options = webdriver.ChromeOptions()
        #self.__options.add_argument('headless')
        #self.__options.add_argument('window-size=1920x1080')
        #self.__options.add_argument('--disable-gpu')
        #self.__options.add_argument("lang=ko_KR") # 한국어!
        #self.__options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
        self.__driver = webdriver.Chrome('C:/Users/user/Downloads/chromedriver_win32/chromedriver')
        #,chrome_options= self.__options
        self.url = "chrome://version/"
    
    def collect_reply_asia(self):
      
        self.__href_list =  self.select_user_company
        print(self.__href_list)
        
        for news in self.__href_list:
            self.__driver.implicitly_wait(3)
            self.__driverr.get(news)
            #self.__driverr.implicitly_wait(30)
            iframes = self.__driver.find_elements_by_css_selector('iframe')
            screenshot_name= "댓글"
            number =1
            
        for iframe in iframes:
                if iframe.get_attribute('title') == 'livere':
                    driver.get(iframe.get_attribute("src"))
                    tt = driver.find_elements_by_css_selector('div')
                    
                    for i in tt:
                        if i.get_attribute('class') == 'reply-content-wrapper':
                            lans = i.get_attribute('data-content')
                            self.__driver.save_screenshot(screenshot_name+str(number)+'.png')
                            number+=1
                            print(i.get_attribute('data-content'))
                    break

    
    @property
    def getdriver(self):
        return self.__driver
    
    def selenium_check(self):
            result = subprocess.call(['pip install selenium'])
            print(result)
    
    
    def chrome_versrion_check(self):
        ##version > span:nth-child(1)
        self.__driver.implicitly_wait(3)
        self.__driver.get(self.url)
        __chrome_version = self.__driver.find_elements_by_tag_name('td')
        #self.__dirver.Quit()
        return __chrome_version[1].text
    
    

In [42]:
#print(Test.chrome_versrion_check())
#Test.selenium_check()  